In [ ]:
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import pyreadstat as prs
import re

from pathlib import Path

import config as c

stataDirs = [c.db2018, 
             c.db2017, 
             c.db2016, 
             c.db2015, 
             c.db2014, 
             c.db2013, 
             c.db2012, 
             c.db2011, 
             c.db2010, 
             c.db2009]

In [ ]:
print(c.db2018)
for f in sorted(os.listdir(c.db2018)):
    print(f)

In [ ]:
for stataDir in stataDirs:
    print('\n')
    print(stataDir)
    cols = 0
    savs = 0
    for f in os.listdir(stataDir):
        if 'deadmen' not in f:
            continue
        filePath = os.path.join(stataDir, f)
        print(f)
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print(df.head())
        #print('count: {} -- {}'.format(len(df.RegNo.unique()), np.sort(df.RegNo.unique()).astype(int)))
        print('number_rows: {}'.format(meta.number_rows))
        print('column_names, n = {}'.format(len(meta.column_names), meta.column_names))
        print('column_labels, n = {}\n'.format(len(meta.column_labels), meta.column_labels))
        colLabels = meta.column_labels
        for l in colLabels:
            print(l)
    break
    print('total savs = {}'.format(savs))
    print('total cols = {}'.format(cols))

## Compare all files in a single year

In [ ]:
for stataDir in stataDirs:
    n=1
    #print('\n{}'.format(stataDir))
    for f in os.listdir(stataDir):
        #if 'deadmen' not in f:
        #    continue
        filePath = os.path.join(stataDir, f)
        print('#{} {:>5s} {}'.format(n, '', f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('{}'.format('-'*40))
        #print(df.info())
        #print('{}'.format('-'*40))
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        i = 1
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            print('{:<3} {:<4} {:<20}  {}'.format(i, unique_cnt, name, label))
            i+=1
    
        n+=1
        #vvl_dict = meta.variable_value_labels
        #for k1, v1 in vvl_dict.items():
        #    print('  {}  (value type: {})'.format(k1, type(v1)))
        #    for k2, v2 in v1.items():
        #        print('    {}  {}'.format(k2, v2))
            
        print('{}\n'.format('-'*80))
    
    break

## Compare a single file across all years

In [ ]:
specific_file = 'sysschedule.sav'
for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    for f in os.listdir(stataDir):
        if specific_file not in f:
            continue
            
        filePath = os.path.join(stataDir, f)
        print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        print(df.head())
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('{}'.format('-'*40))
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        i = 1
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            print('{:<3} {:<6} {:<20}  {}'.format(i, unique_cnt, name, label))
            i+=1
        
        print('\n---- Fields with codes, and their lookup values ---')
        vvl_dict = meta.variable_value_labels
        for k1, v1 in vvl_dict.items():
            print('{:<5}'.format(k1, type(v1)))
            for k2, v2 in v1.items():
                print('  {:<5} {}'.format(k2, v2))
            
        print('{}\n'.format('-'*80))
    break


## Check number of unique regions and their values

In [ ]:
specific_file = 'sysschedule.sav'
for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    for f in os.listdir(stataDir):
        if specific_file not in f:
            continue
            
        filePath = os.path.join(stataDir, f)
        print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))
        
        regions = np.sort(df.RegNo.unique()).astype(int)
        print('Count: {} -- Region Values: {}'.format(len(regions), regions))
        print('{}\n'.format('-'*80))
    
    #break

## Get all questions (long field names) for 2018

In [ ]:
search_year = 2018
i = 1
outCsv = os.path.join(c.outDir, 'fields.csv')
sep = '\t'

try: os.remove(outCsv)
except: Path(outCsv).touch()

for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    if int(year) != search_year:
        continue
    print('yr: {}   I want: {}'.format(year, search_year))

    for f in sorted(os.listdir(stataDir)):
        #if specific_file not in f:
        #    continue
        fname = f.strip('.sav')
        filePath = os.path.join(stataDir, f)
        #print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        #print('{}'.format('-'*40))
        #print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        #print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        
        header = 'Number{0}Year{0}sav file{0}Count of unique values{0}Field (short){0}Field (long)\n'.format(sep)
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            info_string = '{:<2}  {:<4}  {:<6}  {:<6}  {:<20}  {}'.format(i, year, fname, unique_cnt, name, label)
            csv_string = '{1}{0}{2}{0}{3}{0}{4}{0}{5}{0}{6}'.format(sep, i, year, fname, unique_cnt, name, label)
            print(csv_string)
            with open(outCsv, 'a') as f:
                if i==1:
                    f.write(header)
                f.write(csv_string + '\n')
            i+=1

        print('{}\n'.format('-'*80))
        #break

## Bring in Georgia map

In [ ]:
fields = ['Name_sav','Code_sav']
#Read geojson using Geopandas
gdf = gpd.read_file(c.regions, usecols=fields)
gdf.head()


## Look at UID to join all tables into one ...scratch that, setup sysschedule to merge region name to all other tables

In [ ]:
search_year = 2018
tblList = []
sysscheduleCsv = os.path.join(c.outDir, 'sysschedule.csv')
indexFile = 'sysschedule.sav'

dfRegTbl = pd.read_csv(c.regionLookup)

i = 1
for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    if int(year) != search_year:
        continue
    print('yr: {}   I want: {}'.format(year, search_year))
    
    for f in sorted(os.listdir(stataDir)):
        filePath = os.path.join(stataDir, f)
        tblList.append(filePath)

# Read index table (sysschedule), and get UID and Region cols
indexTbl = [t for t in tblList if indexFile in t][0]
df, meta = prs.read_sav(indexTbl)
# replace column names with column labels
df.columns = meta.column_labels

#print('_' * 80)
#print(df.head())
#print('_' * 80)
#print(dfRegTbl)

df = df.merge(dfRegTbl, how='left', left_on='Region', right_on='Code_sav')
df = df.drop(['Code_sav'], axis=1)
df = df.rename(columns={'Name_sav': 'Region Name'})
#print(df.count())
print('_' * 80)

df.to_csv(sysscheduleCsv, sep='\t')
df.head()

#Unique ID of the household in the quarter

## Now add Region Name to all tables and substitute full column names for code names

In [ ]:
search_year = 2018
tblList = []
sysscheduleCsv = os.path.join(c.outDir, 'sysschedule.csv')
joinFld = 'Unique ID of the household in the quarter'
joinFldDrop = 'UID_Drop'


dfIdx = pd.read_csv(sysscheduleCsv, sep='\t')
dfIdx = dfIdx[['Unique ID of the household in the quarter', 
               'Unique ID of the household during the whole period',
               'Region', 
               'Region Name']]
dfIdx = dfIdx.rename(columns={joinFld : joinFldDrop})
#print(dfIdx.head())

for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    if int(year) != search_year:
        continue
    print('yr: {}   I want: {}'.format(year, search_year))
    
    for f in sorted(os.listdir(stataDir)):
        filePath = os.path.join(stataDir, f)
        tblList.append(filePath)

for t in tblList:
    outFile = os.path.join(c.outDir, os.path.basename(t).replace('.sav', '.csv'))
    if 'sysschedule' in t:
        continue
    
    df, meta = prs.read_sav(t)
    # replace column names with column labels
    df.columns = meta.column_labels
    
    if joinFld in meta.column_labels:
        df = df.merge(dfIdx, how='left', left_on=joinFld, right_on=joinFldDrop)
        df = df.drop([joinFldDrop], axis=1)
        df.to_csv(outFile, sep='\t')
        print(df.head())
    else:
        print('⬇ No UID ⬇')
        outFile = outFile.replace('.csv', '_NoUID.csv')
        df.to_csv(outFile, sep='\t') 
        print(df.head())

    print(outFile)
    print('_' * 80)